# Assignment

### TASK 1: Analysis of : Number of products, number of product vs sales, monthly/ annual sales, products sold most and sales made by new or old customers

In [1]:
import pandas as pd
import numpy as np
import warnings as ws
import plotly.graph_objs as go
import plotly.io as pio
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import unicodeit as ut

from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"
pd.options.plotting.backend = "plotly"

ws.filterwarnings("ignore")

commercial_act_df = pd.read_csv("data/commercial_activity_df.csv")
commercial_act_df = commercial_act_df.rename(columns={"Unnamed: 0": "id"})

products_df = pd.read_csv("data/products_df.csv")
products_df = products_df.rename(columns={"Unnamed: 0": "id"})

socio_demo_df = pd.read_csv("data/sociodemographic_df.csv")
socio_demo_df = socio_demo_df.rename(columns={"Unnamed: 0": "id"})

In [2]:
commercial_act_df.head(3)

,id,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,0,1375586,2018-01-28,2018-01-12,KHL,1.0,02 - PARTICULARES
1,1,1050611,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO
2,2,1050612,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO


In [3]:
products_df.head(3)

,id,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


In [4]:
socio_demo_df.head(3)

,id,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11


In [5]:
commercial_prod_temp = commercial_act_df.merge(products_df, on=["pk_cid","pk_partition"],how="inner")
full_commercial_temp_df = commercial_prod_temp.merge(socio_demo_df, on=["pk_cid","pk_partition"],how="inner")
full_commercial_df = full_commercial_temp_df.drop(columns={"id_x","id_y","id"})
full_commercial_df.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,...,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01-28,2018-01-12,KHL,1.0,02 - PARTICULARES,0,0,0,0,...,0,0,0,1,ES,29.0,H,35,N,87218.10
1,1050611,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,13.0,V,23,N,35548.74
2,1050612,2018-01-28,2015-08-10,KHE,0.0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,13.0,V,23,N,122179.11
3,1050613,2018-01-28,2015-08-10,KHD,0.0,03 - UNIVERSITARIO,1,0,0,0,...,0,0,0,0,ES,50.0,H,22,N,119775.54
4,1050614,2018-01-28,2015-08-10,KHE,1.0,03 - UNIVERSITARIO,0,0,0,0,...,0,0,0,1,ES,50.0,V,23,N,NaN


In [6]:
print(f"Size of dataframe: {full_commercial_df.shape[0]}")
print(f"# unique rows: {full_commercial_df.duplicated().shape[0]}")
print(f"# unique pk_cid: {full_commercial_df['pk_cid'].nunique()}")
#print(full_commercial_df.isna().any()) # entry:channel; segment; payroll; pension_plan, region_code; gender; salary

Size of dataframe: 5962924
# unique rows: 5962924
# unique pk_cid: 456373


In [7]:
full_commercial_df.describe()

,pk_cid,active_customer,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,region_code,age,salary
count,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5962924.0,5.962924e+06,5.962863e+06,5.962863e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.962924e+06,5.960660e+06,5.962924e+06,4.450821e+06
mean,1.234930e+06,4.033845e-01,2.581619e-03,7.848498e-05,5.433576e-05,3.370662e-03,3.711770e-03,1.688098e-02,0.0,1.188444e-02,3.451144e-02,3.652641e-02,5.524471e-02,5.483233e-02,9.441140e-02,5.701901e-06,7.348076e-01,2.581440e+01,2.976042e+01,1.158167e+05
std,1.623020e+05,4.905767e-01,5.074401e-02,8.858828e-03,7.371080e-03,5.795948e-02,6.081113e-02,1.288255e-01,0.0,1.083660e-01,1.825388e-01,1.875960e-01,2.284573e-01,2.276527e-01,2.924002e-01,2.387859e-03,4.414356e-01,1.359566e+01,1.198038e+01,1.995519e+05
min,1.589100e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,1.202730e+03
25%,1.112532e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.300000e+01,2.200000e+01,6.150063e+04
50%,1.231097e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.800000e+01,2.500000e+01,8.865465e+04
75%,1.352339e+06,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.600000e+01,3.400000e+01,1.316699e+05
max,1.553689e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.200000e+01,1.050000e+02,2.889440e+07


In [8]:
def show_boxplots(df):

    for col in df.loc[:, ~df.columns.isin(['pk_cid','region_code'])].select_dtypes(include=np.number):
        
        plt.figure(figsize=(15,6))
        sns.boxplot(data=df[col])
        plt.title(str(col))
        plt.show()

#show_boxplots(full_commercial_df)

In [9]:
def show_frequency_tables(df):

    df_cat = df[["entry_channel","segment","country_id",'region_code','gender','deceased']]

    for col in df_cat:
        data = pd.crosstab(df_cat[col], "Count").sort_values(by="Count", ascending=False).reset_index().head(20)
        fig = px.bar(data_frame=data, x=data[col], y=data["Count"], text=data["Count"])
        fig.update_layout(
                    title={
                    'text' : str(col),
                    'x':0.5,
                    'xanchor': 'center'
                })
        fig.update_layout(
            font_family="Courier New",
            title_font_family="Courier New",

        )
        fig.update_xaxes(title_font_family="Courier New", title_font_size=20)
        fig.update_yaxes(title_font_family="Courier New", title_font_size=20)
        fig.update_traces(textfont_size=16, textangle=0, textposition="outside", cliponaxis=False, hovertemplate="%{text}<br>%{x}")

        fig.show()
    
#show_frequency_tables(full_commercial_df)

In [10]:
"""['pk_cid', 'pk_partition', 'entry_date', 'entry_channel', 'active_customer', 'segment', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account','emc_account', 'debit_card', 'em_account_p', 'em_acount', 'country_id','region_code', 'gender', 'age', 'deceased', 'salary']"""

def plot_cat_vs_cat(x, y,df, norm=True, cutoff_count_x=10):
    p_df = (
        df[["pk_cid", x, y]]
        .dropna(how="any", axis=0)
        .pivot_table(index=y, columns=x, values="pk_cid", aggfunc="count")
    )
    p_df = p_df.loc[:, p_df.sum(axis=0) > cutoff_count_x]
    if norm:
        p_df = p_df.div(p_df.sum(axis=0), axis=1) * 100
    
    fig = go.Figure(
        data=[
            go.Bar(
                x=p_df.columns,
                y=p_df.loc[cat],
                name=str(cat),
                showlegend=True
            )
            for cat in np.sort(p_df.index)
        ],
        layout=go.Layout(
            title="{} vs {}".format(y, x),
            barmode="stack"
        )
    )
    fig.show()

boxplots_xs = ["entry_channel","segment","active_customer","short_term_deposit","loans","pension_plan"]
#boxplots_xs = ["entry_channel","segment","mortgage","funds","securities","payroll"]
#boxplots_xs = ["entry_channel","segment","long_term_deposit","em_account_pp","credit_card","payroll_account"]
#boxplots_xs = ["entry_channel","segment","emc_account","debit_card","em_account_p","em_acount"]
#boxplots_xs = ["entry_channel","segment","country_id","region_code","gender","deceased"]
"""for x in boxplots_xs:
    for y in boxplots_xs:
        if y != x:
            plot_cat_vs_cat(x, y, full_commercial_df, norm=True, cutoff_count_x=10)
"""

'for x in boxplots_xs:\n    for y in boxplots_xs:\n        if y != x:\n            plot_cat_vs_cat(x, y, full_commercial_df, norm=True, cutoff_count_x=10)\n'

In [11]:
"""['pk_cid', 'pk_partition', 'entry_date', 'entry_channel', 'active_customer', 'segment', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'em_account_pp',
       'credit_card', 'payroll', 'pension_plan', 'payroll_account','emc_account', 'debit_card', 'em_account_p', 'em_acount', 'country_id','region_code', 'gender', 'age', 'deceased', 'salary']"""

def plot_box_by_cat(x, y, df, norm=True, cutoff_count_x=10):
    fig = go.Figure(
        data=[
            go.Box(
                y=df.loc[df[x] == xval, y],
                name=str(xval),
                showlegend=False
            )
            for xval in np.sort(df[x].dropna().unique())
            if (df[x] == xval).sum() > cutoff_count_x
        ],
        layout=go.Layout(
            title="{} vs {}".format(y, x)
        )
    )
    fig.show()

boxplots_xs = ["active_customer","short_term_deposit","loans","mortgage","funds","em_acount"]
boxplots_xs = ["active_customer"]
#boxplots_xs = ["securities","long_term_deposit","em_account_pp","credit_card","payroll"]
#boxplots_xs = ["pension_plan","payroll_account","emc_account","debit_card","em_account_p"]
boxplots_ys = ["age","salary"] 
"""for x in boxplots_xs:
    for y in boxplots_ys:
        if y != x:
            plot_box_by_cat(x, y, full_commercial_df, norm=True, cutoff_count_x=10)
            """

'for x in boxplots_xs:\n    for y in boxplots_ys:\n        if y != x:\n            plot_box_by_cat(x, y, full_commercial_df, norm=True, cutoff_count_x=10)\n            '